# Purpose: 
Perform XGBoost on Office data

In [2]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 7.2 MB/s eta 0:00:0000:0100:01


In [4]:
import pandas as pd
import re 
import numpy as np 
import os
import re
from tqdm import tqdm
from tqdm.notebook import tqdm
tqdm.pandas()
# import necessary packages for tfidf and statistical models: 
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# XgBoost Model for Disagreement with Ratings:

In [5]:
os.getcwd()

'/home/ec2-user/SageMaker/amazon_thesis_detectron/004_XGBoost'

In [6]:
# read the file: 
disagreement_with_ratings_df = pd.read_csv('/home/ec2-user/SageMaker/amazon_thesis_detectron/001_Process_Manual_Annoted_Files_For_Text_Classification/office_data_version_1/disagreement_with_ratings.csv')
disagreement_with_ratings_df

,review_id,full_review,disagreement_with_ratings,disagreement_with_ratings_ohe
0,R1CRQ4TO7TVZYI,So the image shows noodlers polar ink series a...,0.0,0
1,R19JEPGLD3YJSH,I don’t think I have yet to write a negative r...,0.0,0
2,RITV3I7304IAG,Doesn’t erase. Even after I used some elbow gr...,0.0,0
3,R2C71Z7CM7WQ8O,Sadly I wish I had read the reviews before buy...,0.0,0
4,RGF63NTC51ASI,The media could not be loaded. ...,0.0,0
...,...,...,...,...
4445,R24E9VLZAXT02B,I wouldn't buy this again.1. The instructions...,0.0,0
4446,RRQKDX7BGHW99,I printed approx. 40 9x11 pages of my toolbo...,0.0,0
4447,R1I08PKQ06WQ6W,"Purchased 2/7/15, date of review 3/23/15. Wis...",0.0,0
4448,RRF0HLVMA6TYH,"To echo prior reviews, it is a gamble when you...",0.0,0


In [16]:
def get_train_test_split(df, stratify_col, test_size=0.3, random_state=42): 
    """
    Split the dataframe into training and testing sets.

    Args:
        df (pandas.DataFrame): The input dataframe.
        stratify_col (str or array-like): The column used for stratification during splitting.
        test_size (float, optional): The proportion of the dataset to include in the test split. Default is 0.3.
        random_state (int, optional): Random seed for reproducibility. Default is 42.

    Returns:
        tuple: A df containing the training set and the testing set.
    """
    train, test_df = train_test_split(df, test_size= test_size, random_state=random_state, stratify= stratify_col)
    return train, test_df

def create_xgb_train_data(train,test, text_col, label): 
     """
    Prepare data for training an XGBoost model using TF-IDF features.

    Args:
        train (pandas.DataFrame): The training dataset.
        test (pandas.DataFrame): The testing dataset.
        text_col (str): The name of the column containing the text data.
        label (str): The name of the column containing the labels.

    Returns:
        tuple: A tuple containing the training features, training labels, testing features, and testing labels.
    """
    # tfidf - This approach uses ngrams with a range of 1,3: 
    df_train_xgb = train
    # Convert a collection of raw documents to a matrix of TF-IDF features. This is applied on the train dataset
    # ngram_range is the range for n-gram words to be taken into consideration. Here, unigram, bigram and trigram will be
    # taken into considerataion. 
    # The returned matrix of tf-idf features provides one row for each of the reviews and columns represent the terms.
    tfidf =TfidfVectorizer(ngram_range=(1,3))
    # Learn vocabulary and idf, return document-term matrix in the train dataset.
    x_train = tfidf.fit_transform(df_train_xgb[text_col])
    y_train = df_train_xgb[label]
    # need to only transform the test dataset into document-term features. Here, review-word features. Applying a 
    # transform on the transformed class
    x_test= tfidf.transform(test[text_col])
    y_test= test[label]
    return x_train, y_train, x_test, y_test

In [17]:
# train test split for disagreement with ratings df: 
train, test_df = get_train_test_split(disagreement_with_ratings_df,stratify_col=disagreement_with_ratings_df['disagreement_with_ratings_ohe']
                                       )
x_train, y_train, x_test, y_test = create_xgb_train_data(train,test_df,'full_review', 'disagreement_with_ratings_ohe')

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:1671: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


In [ ]:
param_grid = {
    'learning_rate': [0.1, 0.2, 0.3],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 0.9],
    'colsample_bytree': [0.8, 0.9],
    'n_estimators': [100, 200, 300]
}

xgb = XGBClassifier(objective='binary:logistic', eval_metric='auc')

# Perform grid search using cross-validation
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(x_train, y_train)

# Print the best parameters and best score found during grid search
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)



In [ ]:
# Testing performance metrics: 
y_pred = xgb.predict(x_test)
print(classification_report(y_pred, y_test))

In [18]:
%%time 

#Creating of the XgBoost Classifier taking input as the features returned by the tf-idf transform class from sklearn. 
# n_estimators refers to the number of trees in the classification algorithm. 
# max_depth represents the depth of each individual decision tree in the classification algrothim. Maximum split made
# in the decision trees.
xgb= xgb.XGBClassifier(n_jobs = -1, n_estimators= 4000, max_depth = 2, random_state = 32,learning_rate=0.005, gpu_id = 0
)
xgb.fit(x_train,y_train)

CPU times: user 9min 55s, sys: 2.82 s, total: 9min 58s
Wall time: 39.7 s


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.005, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=4000, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=32, ...)

In [19]:
# Testing performance metrics: 
y_pred = xgb.predict(x_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       1.00      0.97      0.98      1328
           1       0.08      0.57      0.14         7

    accuracy                           0.96      1335
   macro avg       0.54      0.77      0.56      1335
weighted avg       0.99      0.96      0.98      1335



In [21]:
%%time 

import xgboost as xgb
xgb= xgb.XGBClassifier(n_jobs = -1, n_estimators= 2000, max_depth = 2, random_state = 32,learning_rate=0.005, gpu_id = 0
)
xgb.fit(x_train,y_train)

CPU times: user 4min 59s, sys: 1.22 s, total: 5min
Wall time: 18.8 s


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.005, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=2000, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=32, ...)

In [22]:
# Testing performance metrics: 
y_pred = xgb.predict(x_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98      1332
           1       0.00      0.00      0.00         3

    accuracy                           0.96      1335
   macro avg       0.50      0.48      0.49      1335
weighted avg       1.00      0.96      0.98      1335



# XGBoost Model for Rating Management Explicit: 

In [24]:
# read the file: 
rating_management_explicit_df = pd.read_csv('/home/ec2-user/SageMaker/amazon_thesis_detectron/001_Process_Manual_Annoted_Files_For_Text_Classification/office_data_version_1/rating_managment_explicit_v1.csv')
rating_management_explicit_df

,review_id,full_review,rating_managment_explicit,rating_managment_explicit_ohe
0,R1CRQ4TO7TVZYI,So the image shows noodlers polar ink series a...,0,0
1,R19JEPGLD3YJSH,I don’t think I have yet to write a negative r...,0,0
2,RITV3I7304IAG,Doesn’t erase. Even after I used some elbow gr...,1,1
3,R2C71Z7CM7WQ8O,Sadly I wish I had read the reviews before buy...,0,0
4,RGF63NTC51ASI,The media could not be loaded. ...,0,0
...,...,...,...,...
4445,R24E9VLZAXT02B,I wouldn't buy this again.1. The instructions...,0,0
4446,RRQKDX7BGHW99,I printed approx. 40 9x11 pages of my toolbo...,0,0
4447,R1I08PKQ06WQ6W,"Purchased 2/7/15, date of review 3/23/15. Wis...",0,0
4448,RRF0HLVMA6TYH,"To echo prior reviews, it is a gamble when you...",0,0


In [25]:
# train test split for disagreement with ratings df: 
train, test_df = get_train_test_split(rating_management_explicit_df,stratify_col=rating_management_explicit_df['rating_managment_explicit_ohe']
                                       )
x_train, y_train, x_test, y_test = create_xgb_train_data(train,test_df,'full_review', 'rating_managment_explicit_ohe')

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:1671: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


In [31]:
%%time 

import xgboost as xgb
xgb= xgb.XGBClassifier(n_jobs = -1, n_estimators= 7000, max_depth = 2, random_state = 32,learning_rate=0.05, gpu_id = 0
)
xgb.fit(x_train,y_train)

CPU times: user 15min 11s, sys: 5.98 s, total: 15min 17s
Wall time: 58 s


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=7000, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=32, ...)

In [32]:
# Testing performance metrics: 
y_pred = xgb.predict(x_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1330
           1       0.05      0.20      0.08         5

    accuracy                           0.98      1335
   macro avg       0.52      0.59      0.53      1335
weighted avg       0.99      0.98      0.99      1335



# XgBoost Model for Wish I had read Reviews:

In [33]:
read_reviews_df = pd.read_csv('/home/ec2-user/SageMaker/amazon_thesis_detectron/001_Process_Manual_Annoted_Files_For_Text_Classification/office_data_version_1/read_reviews_v1.csv')
read_reviews_df

,review_id,full_review,wish I read reviews,wish I read reviews _ohe
0,RZQSEN3KXZJ12,Not happy with my black ink cartridge I got ma...,0,0
1,R3CL4QCWRXZEIX,I only hope that this was a quality assurance ...,0,0
2,R35NRB4TB7S6J4,I generally don't review the things I buy but ...,0,0
3,R2N024W2JDX6B0,I'm giving a single star because I have to ret...,0,0
4,R6WWRPSUQQQ97,I read a couple other reviews on how they have...,0,0
...,...,...,...,...
4791,R24E9VLZAXT02B,I wouldn't buy this again.1. The instructions...,0,0
4792,RRQKDX7BGHW99,I printed approx. 40 9x11 pages of my toolbo...,0,0
4793,R1I08PKQ06WQ6W,"Purchased 2/7/15, date of review 3/23/15. Wis...",1,1
4794,RRF0HLVMA6TYH,"To echo prior reviews, it is a gamble when you...",0,0


In [35]:
# train test split for disagreement with ratings df: 
train, test_df = get_train_test_split(read_reviews_df,stratify_col=read_reviews_df['wish I read reviews _ohe']
                                       )
x_train, y_train, x_test, y_test = create_xgb_train_data(train,test_df,'full_review', 'wish I read reviews _ohe')

In [37]:
%%time 

import xgboost as xgb
xgb= xgb.XGBClassifier(n_jobs = -1, n_estimators= 7000, max_depth = 2, random_state = 32,learning_rate=0.005, gpu_id = 0
)
xgb.fit(x_train,y_train)

CPU times: user 20min 3s, sys: 6.81 s, total: 20min 10s
Wall time: 1min 16s


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.005, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=7000, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=32, ...)

In [38]:
# Testing performance metrics: 
y_pred = xgb.predict(x_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.98      0.97      0.97      1345
           1       0.61      0.74      0.67        94

    accuracy                           0.95      1439
   macro avg       0.80      0.86      0.82      1439
weighted avg       0.96      0.95      0.95      1439



#  XgBoost Model for Wrong Buying 

In [39]:
wrong_buying_df = pd.read_csv('/home/ec2-user/SageMaker/amazon_thesis_detectron/001_Process_Manual_Annoted_Files_For_Text_Classification/office_data_version_1/wrong_buying_v1.csv')
wrong_buying_df

,review_id,full_review,wrong_buying,wrong_buying_ohe
0,R1CRQ4TO7TVZYI,So the image shows noodlers polar ink series a...,0.0,0
1,R19JEPGLD3YJSH,I don’t think I have yet to write a negative r...,0.0,0
2,RITV3I7304IAG,Doesn’t erase. Even after I used some elbow gr...,0.0,0
3,R2C71Z7CM7WQ8O,Sadly I wish I had read the reviews before buy...,0.0,0
4,RGF63NTC51ASI,The media could not be loaded. ...,0.0,0
...,...,...,...,...
4445,R24E9VLZAXT02B,I wouldn't buy this again.1. The instructions...,0.0,0
4446,RRQKDX7BGHW99,I printed approx. 40 9x11 pages of my toolbo...,0.0,0
4447,R1I08PKQ06WQ6W,"Purchased 2/7/15, date of review 3/23/15. Wis...",0.0,0
4448,RRF0HLVMA6TYH,"To echo prior reviews, it is a gamble when you...",0.0,0


In [40]:
# train test split for disagreement with ratings df: 
train, test_df = get_train_test_split(wrong_buying_df,stratify_col=wrong_buying_df['wrong_buying_ohe']
                                       )
x_train, y_train, x_test, y_test = create_xgb_train_data(train,test_df,'full_review', 'wrong_buying_ohe')

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:1671: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


In [42]:
%%time 

import xgboost as xgb
xgb= xgb.XGBClassifier(n_jobs = -1, n_estimators= 6500, max_depth = 2, random_state = 32,learning_rate=0.08, gpu_id = 0
)
xgb.fit(x_train,y_train)

CPU times: user 14min 45s, sys: 3.9 s, total: 14min 49s
Wall time: 55.7 s


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.08, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=6500, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=32, ...)

In [43]:
# Testing performance metrics: 
y_pred = xgb.predict(x_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.98      0.93      0.95      1266
           1       0.35      0.68      0.46        69

    accuracy                           0.92      1335
   macro avg       0.66      0.81      0.71      1335
weighted avg       0.95      0.92      0.93      1335



# XGBoost for zero stars: 

In [44]:
zero_stars_df = pd.read_csv('/home/ec2-user/SageMaker/amazon_thesis_detectron/001_Process_Manual_Annoted_Files_For_Text_Classification/office_data_version_1/zero_star_v1.csv')
zero_stars_df

,review_id,full_review,zero_star,zero_star_ohe
0,R1CRQ4TO7TVZYI,So the image shows noodlers polar ink series a...,0,0
1,R19JEPGLD3YJSH,I don’t think I have yet to write a negative r...,0,0
2,RITV3I7304IAG,Doesn’t erase. Even after I used some elbow gr...,0,0
3,R2C71Z7CM7WQ8O,Sadly I wish I had read the reviews before buy...,0,0
4,RGF63NTC51ASI,The media could not be loaded. ...,4,1
...,...,...,...,...
4445,R24E9VLZAXT02B,I wouldn't buy this again.1. The instructions...,0,0
4446,RRQKDX7BGHW99,I printed approx. 40 9x11 pages of my toolbo...,1,1
4447,R1I08PKQ06WQ6W,"Purchased 2/7/15, date of review 3/23/15. Wis...",0,0
4448,RRF0HLVMA6TYH,"To echo prior reviews, it is a gamble when you...",0,0


In [45]:
# train test split for zero stars df: 
train, test_df = get_train_test_split(zero_stars_df,stratify_col=zero_stars_df['zero_star_ohe']
                                       )
x_train, y_train, x_test, y_test = create_xgb_train_data(train,test_df,'full_review', 'zero_star_ohe')

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:1671: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


In [47]:
%%time 

import xgboost as xgb
xgb= xgb.XGBClassifier(n_jobs = -1, n_estimators= 6500, max_depth = 2, random_state = 32,learning_rate=0.007, gpu_id = 0
)
xgb.fit(x_train,y_train)

CPU times: user 16min 39s, sys: 5.16 s, total: 16min 45s
Wall time: 1min 3s


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.007, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=6500, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=32, ...)

In [48]:
# Testing performance metrics: 
y_pred = xgb.predict(x_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97      1167
           1       0.73      0.85      0.79       168

    accuracy                           0.94      1335
   macro avg       0.86      0.90      0.88      1335
weighted avg       0.95      0.94      0.94      1335



In [49]:
print("End of Notebook")

End of Notebook
